In [4]:
import numpy as np
import import_ipynb
from chapter1 import solveUpper, solveLower, CholeskyDecomp
from matplotlib import pyplot as plt

plt.rcdefaults()
plt.style.use("seaborn-whitegrid")
plt.rc("figure", figsize=(11.2, 6.3))
plt.rc("font", size=12)
plt.rc("axes", edgecolor="white")
plt.rc("legend", frameon=True, framealpha=0.8, facecolor="white", edgecolor="white")

# 2 Numerical Linear Algebra II

## Problem Statement
For $A\in\mathbb{K}^{n\times m}$ and $\ b\in\mathbb{K}^n$ find $x\in\mathbb{K}^m$ such that $Ax=b,$ where $\mathbb{K}=\mathbb{R}$ or $\mathbb{C}.$

If $n<m$ the system is called _underdetermined_ and there exist infinitely many solutions. If on the other hand $n>m$ then the system is _overdetermined_ and there may not even be a solution, i.e. if $b\notin\textrm{im}(A)$ since $\dim\textrm{im}(A) = m.$

For an _overdetermined_ system $Ax=b,\ x\in\mathbb{K}^m$ is called a _solution_ if $|\!|Ax-b|\!| = \min\{|\!|Ay-b|\!|\ :\ y \in \mathbb{K}^m\}.$

For $|\!|\cdot|\!|_2$ such a solution is called _least squares_ and for $|\!|\cdot|\!|_\infty$ _Chebyshev approximation._

## Normal Equation
Solving $AA^*x=A^*b$ yields the least squares solution if $A$ has full rank.

In [5]:
def solveNormalEquation(A, b):
    '''O(n^2 (m+n)/3)'''
    A = np.array(A, dtype = np.double)
    H = A.T.conj() @ A
    c = A.T.conj() @ b
    if H.shape == ():
        H = [[H]]
        c = [c]
    L = CholeskyDecomp(H)
    y = solveLower(L, c)
    return solveUpper(L.T.conj(), y)

In [6]:
A = [[12,-51,4],[6,167,-68],[-4,24,-41],[10,8,38]]
b = [1,1,1,1]
x = solveNormalEquation(A, b)
print(np.abs(A @ x - b))

[0.12041078 0.18921694 0.86007702 0.60205392]


## QR Decomposition
reduced QR decomposition: $A = \tilde{Q}\tilde{R}$ for $\tilde{Q}\in\mathbb{K}^{n\times m}$ a matrix with orthonormal columns and $\tilde{R}\in\mathbb{K}^{m\times m}$ upper triangle with positive diagonal elements

(full) QR decomposition: $A = QR$ for $Q\in\textrm{U}(n)$ and $R\in\mathbb{K}^{n\times m}$ a generalized upper triangle matrix, meaning an $m\times m$ upper triangle matrix with $n-m$ rows of zeros at the end.

### QR Decomposition via Gram-Schmidt

In [7]:
def norm2(x):
    return np.sqrt(np.abs(x.conj() @ x))

def GramSchmidtQR(A):
    '''O(nm^2)'''
    
    A = np.array(A, dtype = np.double)
    n, m = A.shape
    R = np.zeros((m,m), dtype = np.double)
    Q = np.zeros((n,m), dtype = np.double)
    
    #loops over columns
    for j in range(m):
        
        #fills in strict upper triangle
        for i in range(j):
            R[i,j] = Q[:,i].conj().T @ A[:,j]
            
        v = A[:,j] - (R[:j,j] * Q[:,:j]).sum(axis=1)
        R[j,j] = norm2(v)
        Q[:,j] = v / R[j,j]
    
    return Q, R

In [8]:
A = [[12,-51,4],[6,167,-68],[-4,24,-41],[10,8,38]]
Q, R = GramSchmidtQR(A)
print(Q@R - A)

[[0.00000000e+00 0.00000000e+00 1.77635684e-15]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.77635684e-15 0.00000000e+00 0.00000000e+00]]


In [9]:
def leastSquares(A, b):
    '''O(nm^2)'''
    Q, R = GramSchmidtQR(A) #O(nm^2)
    return solveUpper(R, Q.T.conj() @ b) #O(m^2)

In [14]:
A = [[12,-51,4],[6,167,-68],[-4,24,-41],[10,8,38]]
b = [1,1,1,1]
x = leastSquares(A, b)
print(np.abs(A @ x - b))

[0.12041078 0.18921694 0.86007702 0.60205392]


### QR Decomposition via Householder Transformation
The Householder transformation describes a reflection of a vector about a hyperplane that contains the origin. For a unit vector $v\in\mathbb{K}^n$ that is orthogonal to the hyperplane the corresponding reflection matrix $P_v$ is given by $P_v = I - 2vv^* \in\mathbb{K}^{n \times n}.$

In [31]:
def HouseholderQR(A):
    '''without Q reconstruction: 2nm^2 - 2/3 m^3 flops'''
    R = np.array(A, dtype = np.double)
    n, m = R.shape
    Q = np.eye(n, dtype = np.double)
    I = np.eye(n, dtype = np.double)
    
    for k in range(m - (n==m)):
        a = np.zeros(n)
        a[k:] = R[k:,k]
        vt = a - norm2(a) * np.copysign(I[k],a[k])
        v = vt / norm2(vt)
        H = I - 2 * np.dot(v[:,None], v[None,:])
        R = H @ R
        Q = Q @ H #can be more effieciently reconstructed from the vectors v 
    
    return Q, R

In [33]:
A = [[12,-51,4],[6,167,-68],[-4,24,-41],[10,8,38]]
Q, R = HouseholderQR(A)
print(Q@R - A)

[[-1.77635684e-15 -2.84217094e-14  0.00000000e+00]
 [-8.88178420e-16  2.84217094e-14  0.00000000e+00]
 [ 8.88178420e-16  0.00000000e+00  7.10542736e-15]
 [ 0.00000000e+00  0.00000000e+00  7.10542736e-15]]


## Givens Rotations
Similarly to the Householder transformation we can introduce zeros in $A$ to transform it into the upper triangle matrix $R$ by using a rotations in the $(i,j)$-plane of $\theta$ radians. For $c=\cos(\theta)$ and $s=\sin(\theta)$ the corresponding rotation matrix is given by
$$G(i,j,\theta) = \begin{pmatrix} 1      & \dots  & 0      & \dots  & 0      & \dots  & 0      \\
\vdots & \ddots & \vdots &        & \vdots &        & \vdots \\
0      & \dots  & c      & \dots  & -s     & \dots  & 0      \\
\vdots &        & \vdots & \ddots & \vdots &        & \vdots \\
0      & \dots  & s      & \dots  & c      & \dots  & 0      \\
\vdots &        & \vdots &        & \vdots & \ddots & \vdots \\
0      & \dots  & 0      & \dots  & 0      & \dots  & 1    \end{pmatrix}$$

In [34]:
def GivensRotation(a, b):
    '''6m flops'''
    if b == 0:
        c = np.copysign(a,1)
        s = 0
    elif a == 0:
        c = 0
        s = np.sign(b)
    elif np.abs(a) > np.abs(b):
        t = b / a
        u = np.sign(a) * np.sqrt(1 + t**2)
        c = 1 / u
        s = c * t
    else:
        t = a / b
        u = np.sign(b) * np.sqrt(1 + t**2)
        s = 1 / u
        c = s * t
    
    G = np.array([[c,s],[-s,c]])
    return G

def GivensQR(A):
    '''3nm^2 - m^3 flops'''
    R = np.array(A, dtype = np.double)
    n, m = R.shape
    Q = np.eye(n)
    
    for j in range(n):
        for i in reversed(range(j+1,n)):
            G = GivensRotation(R[i-1,j], R[i,j])
            R[i-1:i+1,j:] = G @ R[i-1:i+1,j:]
            Q[:,i-1:i+1] = Q[:,i-1:i+1] @ G.T
    return Q, R

In [35]:
A = [[12,-51,4],[6,167,-68],[-4,24,-41],[10,8,38]]
Q, R = GivensQR(A)
print(Q@R - A)

[[-1.77635684e-15 -2.13162821e-14  0.00000000e+00]
 [ 0.00000000e+00  5.68434189e-14 -2.84217094e-14]
 [ 8.88178420e-16 -7.10542736e-15  1.42108547e-14]
 [-1.77635684e-15  5.32907052e-15  0.00000000e+00]]


## Singular Value Decomposition
The image of the unit sphere $S_n$ by $A\in\mathbb{K}^{n\times m}$ is an ellipsoid and the lengths $\sigma_1,\dots,\sigma_n$ of its semi-axes are called _singular values._ The corresponding unit vectors $u_i\mathbb{K}^m$ are called _left-singular vectors_ and their preimages $v_i = A^{-1}(\sigma_i u_i)\in\mathbb{K}^m$ _right-singular vectors._

In matrix notation we can write $AV = \hat{U}\hat{\Sigma}$ for $\hat{\Sigma}=\textrm{diag}(\sigma_1,\dots,\sigma_n)\in\mathbb{K}^{n\times n}, \hat{U}=(u_1 | \dots | u_n)\in\mathbb{K}^{n \times m}$ and $V=(v_1 | \dots | v_n)\in\mathbb{K}^{m \times m}.$ The _reduced singular value decomposition_ is $A = \hat{U}\hat{\Sigma}V^*$ since $V\in\textrm{U}(m).$

By extending $\hat{U}$ to $U\in\textrm{U}(n)$ and $\hat{\Sigma}$ to a generalized diagonal matrix $\Sigma\in\mathbb{K}^{n\times m}$ with nullrows we get the _(full) singular value decomposition_ $A=U\Sigma V^*$ which exists for every matrix.



In [26]:
def leastSquaresSVD(A, b):
    U, s, Vh = np.linalg.svd(A, full_matrices=False)
    y = (U.conj().T @ b) / s
    return Vh.conj().T @ y

In [38]:
A = [[12,-51,4],[6,167,-68],[-4,24,-41],[10,8,38]]
b = [1,1,1,1]
print(np.abs(A @ leastSquaresSVD(A, b) - b))

[0.12041078 0.18921694 0.86007702 0.60205392]
